In [1]:
import tkinter as tk
from tkinter import messagebox
%pip install pyspellchecker
from spellchecker import SpellChecker
import sqlite3

# Function to create a database and table (if not already exists)
def create_db():
    conn = sqlite3.connect('spell_checker.db')  # Connect to the database file (or create it)
    cursor = conn.cursor()
    
    # Create a table if it doesn't already exist
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS spell_history (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        original_word TEXT NOT NULL,
        corrected_word TEXT NOT NULL
    )''')
    conn.commit()  # Commit changes
    conn.close()

# Function to insert data into the database
def insert_to_db(original_word, corrected_word):
    conn = sqlite3.connect('spell_checker.db')  # Connect to the database
    cursor = conn.cursor()
    
    # Insert the word and its correction into the database
    cursor.execute('''
    INSERT INTO spell_history (original_word, corrected_word)
    VALUES (?, ?)''', (original_word, corrected_word))
    
    conn.commit()  # Commit changes
    conn.close()

# Function to fetch data from the database and display it
def fetch_history():
    conn = sqlite3.connect('spell_checker.db')  # Connect to the database
    cursor = conn.cursor()
    
    # Fetch all the records from the spell_history table
    cursor.execute('SELECT * FROM spell_history ORDER BY id DESC')
    rows = cursor.fetchall()
    
    # Display the records in the history Text widget with serial numbers
    history.delete(1.0, tk.END)  # Clear the history Text widget before inserting new data
    
    for idx, row in enumerate(rows, start=1):
        original_word, corrected_word = row[1], row[2]
        history.insert(tk.END, f"{idx}. {original_word} -> {corrected_word}\n")
    
    conn.close()

# Function to reset (clear) the history and database entries
def reset_history():
    # Clear the Text widget
    history.delete(1.0, tk.END)
    
    # Remove all records from the database
    conn = sqlite3.connect('spell_checker.db')
    cursor = conn.cursor()
    cursor.execute('DELETE FROM spell_history')  # Delete all records from the table
    conn.commit()
    conn.close()

    messagebox.showinfo("Reset", "All history entries have been deleted.")

# Function to correct the word and show the result
def correct_word(event=None):  # Allow event parameter for key press
    word = entry.get()  # Get input from the user
    if word:
        spell = SpellChecker()  # Initialize the SpellChecker
        corrected_word = spell.correction(word)  # Get the corrected word
        
        # Show the corrected word in the result label
        result_text = f"Original word: {word} -> Corrected word: {corrected_word}\n"
        
        # Display the result in the result label
        result_label.config(text=f"Corrected word will appear here.")
        
        # Insert the word and its correction into the database
        insert_to_db(word, corrected_word)
        
        # Refresh the history to show updated list from the database
        fetch_history()
        
        # Clear the entry box after hitting Enter
        entry.delete(0, tk.END)
    else:
        messagebox.showwarning("Input Error", "Please enter a word.")

# Set up the main window
root = tk.Tk()
root.title("Spell Checker")

# Create the database and table if not exists
create_db()

# Create a label for instructions
instruction_label = tk.Label(root, text="Enter a word to check its spelling:")
instruction_label.pack(pady=10)

# Create an entry widget for user input with center alignment and larger size
entry = tk.Entry(root, width=50, justify=tk.CENTER, font=("Helvetica", 14))  # Larger box with larger text
entry.pack(pady=10)

# Create a button to trigger the correction
check_button = tk.Button(root, text="Check Spelling", command=correct_word, font=("Helvetica", 12))
check_button.pack(pady=10)

# Create a label to display the result
result_label = tk.Label(root, text="Corrected word will appear here.", font=("Helvetica", 12))
result_label.pack(pady=10)

# Create a Text widget to display previous corrections with reduced horizontal padding
history = tk.Text(root, height=10, width=60, font=("Helvetica", 12), padx=5)
history.pack(pady=10)

# Create a Reset button to delete all entries
reset_button = tk.Button(root, text="Reset History", command=reset_history, font=("Helvetica", 12))
reset_button.pack(pady=10)

# Fetch and display previous spell checks from the database
fetch_history()

# Bind the Enter key to trigger the spell check
root.bind("<Return>", correct_word)  # This binds the Enter key to the correct_word function

# Run the application
root.mainloop()
